<a href="https://colab.research.google.com/github/Challakrupajyothi/Rag-Pipeline/blob/main/CHAT_WITH_PDF_USING_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CHAT WITH PDF USING RAG Pipeline**

Installing the Required libraries


In [ ]:
!git clone https://github.com/Challakrupajyothi/Rag-Pipeline.git

Cloning into 'Rag-Pipeline'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [ ]:
pip install PyPDF2 langchain openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install langchain-community


In [ ]:
pip install tiktoken


In [ ]:
import os

# Define directory path
pdf_dir = "/content/tables.pdf"

# Create the directory
os.makedirs(pdf_dir, exist_ok=True)

print(f"Directory created: {pdf_dir}")


Directory created: /content/tables.pdf


In [ ]:
pip install openai==0.28

In [ ]:
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import openai
#USE YOUR OWN OPEN AI KEY
openai.api_key = ""
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

# This Function is used to take the PDF
def extract_text_from_pdf(pdf_path):
    text_chunks = []
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text = page.extract_text()
            if text and text.strip():
                text_chunks.append(text)
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text_chunks

# The PDF is divided into Chunks using Embedding Model
def process_and_embed_chunks(file_path):
    text_chunks = extract_text_from_pdf(file_path)
    embeddings = []
    for chunk in text_chunks:
        embedding = embedding_model.encode(chunk).tolist()
        embeddings.append({"text": chunk, "embedding": embedding})
    return embeddings

# The Prompt Give By user is Evaluated Here
def query_pipeline(query, embeddings):
    query_embedding = embedding_model.encode(query).tolist()

    results = sorted(
        embeddings,
        key=lambda x: cosine_similarity(query_embedding, x["embedding"]),
        reverse=True
    )[:5]  # Top 5 results

    # Here the ai model has used
    context = "\n\n".join([result["text"] for result in results])
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an assistant that answers questions based on provided context."},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"}
        ]
    )
    return response["choices"][0]["message"]["content"]

def cosine_similarity(vec1, vec2):
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    norm_a = sum(a * a for a in vec1) ** 0.5
    norm_b = sum(b * b for b in vec2) ** 0.5
    return dot_product / (norm_a * norm_b)

pdf_dir = "/content/tables.pdf"
all_embeddings = []
for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_dir, filename)
        all_embeddings.extend(process_and_embed_chunks(file_path))

# Query by the User
user_query = input("Enter your query\t")
response = query_pipeline(user_query, all_embeddings)
print(response)


Enter your query	Table of Yearly U.S. GDP by Industry (in millions of dollars)
The table of Yearly U.S. GDP by Industry (in millions of dollars) provides the following data for the years 2010 to 2015:

- **All Industries**:
  - 2010: 26,093,515
  - 2011: 27,535,971
  - 2012: 28,663,246
  - 2013: 29,601,191
  - 2014: 30,895,407
  - 2015: 31,397,023

- **Manufacturing**:
  - 2010: 4,992,521
  - 2011: 5,581,942
  - 2012: 5,841,608
  - 2013: 5,953,299
  - 2014: 6,047,477
  - 2015: 5,829,554

- **Finance, Insurance, Real Estate, Rental, Leasing**:
  - 2010: 4,522,451
  - 2011: 4,618,678
  - 2012: 4,797,313
  - 2013: 5,031,881
  - 2014: 5,339,678
  - 2015: 5,597,018

- **Arts, Entertainment, Recreation, Accommodation, and Food Service**:
  - 2010: 964,032
  - 2011: 1,015,238
  - 2012: 1,076,249
  - 2013: 1,120,496
  - 2014: 1,189,646
  - 2015: 1,283,813

- **Other Industries**:
  - 2010: 15,614,511
  - 2011: 16,320,113
  - 2012: 16,948,076
  - 2013: 17,495,515
  - 2014: 18,318,606
  - 2015: 